## Modèle BERT

In [1]:
from helpers import *

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]  
df = pd.read_csv("./../input/training.1600000.processed.noemoticon.csv", header=None, names=cols)

# df = df.sample(n=1000, random_state=42)
len_df = len(df)
print(df.shape)
df.head()

(1600000, 6)


,sentiment,timestamp,date,query,username,comment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 1600000/1600000 [00:03<00:00, 405816.13it/s]


(1469990, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,awww thats a bummer you shoulda got david carr...,19,16
1,is upset that he can't update his Facebook by ...,0,is upset that he cant update his facebook by t...,21,21
2,@Kenichan I dived many times for the ball. Man...,0,i dived many times for the ball managed to sav...,18,16
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire,10,10
4,"@nationwideclass no, it's not behaving at all....",0,no its not behaving at all im mad why am i her...,21,20


- les données sont préparées (input ids, attention mask) pour mettre en oeuvre une démarche d’embedding pour le “modèle BERT”
- une démarche optionnelle d’embedding via USE est mise en oeuvre.


In [5]:
from transformers import BertTokenizer, TFBertForSequenceClassification  
from transformers import InputExample, InputFeatures
from transformers import BertTokenizerFast
import tensorflow as tf  
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

/Users/m/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')  


In [7]:
# Assume preprocess_text is defined elsewhere  
df_bert_approach = df.copy()

In [8]:
df_bert_approach = df_bert_approach.reset_index(drop=True)
X_train, X_test, y_train, y_test = train_test_split(df_bert_approach["comment_clean"], df_bert_approach['sentiment'], test_size=0.3, random_state=42)  
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)  

In [9]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)  
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)  

In [10]:
class TweetDataset(Dataset):  
    def __init__(self, encodings, labels=None):  
        self.encodings = encodings  
        self.labels = labels  
  
    def __getitem__(self, idx):  
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}  
        if self.labels:  
            item['labels'] = torch.tensor(self.labels[idx])  
        return item  
  
    def __len__(self):  
        return len(self.encodings.input_ids)  

In [11]:
# Convert to torch Dataset  
train_dataset = TweetDataset(train_encodings, y_train.tolist())  
val_dataset = TweetDataset(val_encodings, y_val.tolist())

In [12]:
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)  
test_dataset = TweetDataset(test_encodings, y_test.tolist())

In [13]:
# Load DistilBERT model  
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(df_bert_approach['sentiment'].tolist())))  

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# !pip install accelerate -U

In [15]:
# Define training arguments  
training_args = TrainingArguments(  
  output_dir='./results',          # output directory  
  num_train_epochs=1,              # total number of training epochs  
  per_device_train_batch_size=256,   # batch size per device during training, adjust based on your GPU  
  per_device_eval_batch_size=256,   # batch size for evaluation  
  warmup_steps=500,                # number of warmup steps for learning rate scheduler  
  weight_decay=0.01,               # strength of weight decay  
  logging_dir='./logs',            # directory for storing logs  
  logging_steps=10,  
  evaluation_strategy="epoch",  
  save_strategy="epoch",           # Ensure the save strategy matches the evaluation strategy  
  load_best_model_at_end=True,  
)  

In [16]:
# Initialize the Trainer  
trainer = Trainer(  
  model=model,  
  args=training_args,  
  train_dataset=train_dataset,  
  eval_dataset=val_dataset  
)  

In [17]:
import torch

In [18]:
# Train the model  
trainer.train()  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 10/4020 [00

{'loss': 0.696, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/4020 [01:11<3:44:41,  3.37s/it]

{'loss': 0.6943, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  1%|          | 30/4020 [01:47<3:59:57,  3.61s/it]

...

 10%|▉         | 390/4020 [27:45<4:42:14,  4.67s/it]

{'loss': 0.4152, 'learning_rate': 3.9000000000000006e-05, 'epoch': 0.1}


 10%|▉         | 400/4020 [28:32<4:41:01,  4.66s/it]

{'loss': 0.4137, 'learning_rate': 4e-05, 'epoch': 0.1}


 10%|█         | 409/4020 [29:11<4:21:23,  4.34s/it]

In [ ]:
predictions = trainer.predict(test_dataset) 

In [ ]:
predictions

In [ ]:
from scipy.special import softmax  

# Getting the raw predictions  
raw_predictions = predictions.predictions  
  
# Assuming a classification task, apply softmax if needed (for getting probabilities)  
prob_predictions = softmax(raw_predictions, axis=1)  
  
# For getting the predicted class indices  
predicted_classes = np.argmax(prob_predictions, axis=1)  

In [ ]:
accuracy = accuracy_score(y_test, predicted_classes)
print(accuracy)